### Note: if run manually, the `mediatel_date` argument should always be set to the beginning of the week on Monday in `yyyymmdd` string format.

In [0]:
%run ./my_packages

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


# Daily functions

# Weekly functions

# Monthly functions

In [0]:
# Set `mediatel_date` here if you want to run manually. Otherwise under BAU conditions (i.e. when you don't need to run any past dates) this cell should be skipped via %skip tag.
start = '20260216'
end = '20260222'
mediatel_dates = [
    (datetime.strptime(start, "%Y%m%d") + timedelta(days=i)).strftime("%Y%m%d") for i in range((datetime.strptime(end, "%Y%m%d") - datetime.strptime(start, "%Y%m%d")).days + 1)
    ]

In [0]:
for mediatel_date in mediatel_dates:
    # Read in latest Mediatel data and transform
    df_mediatel = f_LoadLatestMediatel(mediatel_date, select_cols=select_cols)
    df_mediatel = f_WeeklyCreateWeekNum(df_mediatel)

    # Read in current buildup file else create one if it doesn't exist
    buildup_file_name = "./Output/brett_collections_funnel_buildup_weekly.parquet"
    if os.path.isfile(buildup_file_name):
        # Buildup file exists; load it
        df_buildup = pl.read_parquet(buildup_file_name)

        # Check if buildup file has 7 days worth of data
        buildup_len = len(df_buildup.select("call_date").unique())

        # If <7 days worth of extracts then continue building up
        if buildup_len >= 1 and buildup_len <= 5:
            print("Buildup file has less than 7 days worth of data. Continue building up.")
            # Stack latest mediatel data to buildup table
            df_buildup = pl.concat([df_buildup, df_mediatel], how="vertical")
            # Overwrite buildup table
            df_buildup.write_parquet(buildup_file_name)

        elif buildup_len == 6:
            # After adding 7th extract it will be complete
            print("Buildup file has 7 days worth of data. Trigger summary process.")
            df_buildup = pl.concat([df_buildup, df_mediatel], how="vertical")
            # Metrics should only be created at the end of each week prior to appending to main weekly summary table
            df_buildup = f_CreateMetrics(df_buildup, timeframe="weekly")
            df_buildup_summary = f_CreateSummary(df_buildup, "weekly")
            df_buildup_summary = df_buildup_summary.to_pandas()
            # Append now summarised buildup table to weekly summary
            table_name = "teams.cashalo.brett_collections_funnel_weekly_test"
            weekly_summary = spark.sql(f"SELECT * FROM {table_name}").toPandas()
            weekly_summary = pd.concat([weekly_summary, df_buildup_summary], ignore_index=True)
            # Save new weekly summary
            spark_df = f_WeeklyCreateSparkTable(weekly_summary, table_name=table_name)
            spark_df.write.saveAsTable(f"{table_name}")
            # Delete buildup table so that a new buildup table is created at the beginning of next week
            file_path = Path(buildup_file_name)
            file_path.unlink()

        else:
            dbutils.notebook.exit("More than 7 days worth of data, check buildup file or Mediatel extract.")
            
    else:
        # Buildup file doesn't exist; use df_mediatel to initiate it.
        print("Build up file initiated")
        df_mediatel.write_parquet(buildup_file_name)

Build up file initiated
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has 7 days worth of data. Trigger summary process.
Build up file initiated
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has 7 days worth of data. Trigger summary process.
Build up file initiated
Buildup file has less than 7 days worth of data. Continue building up.
Buildup file has le

# Checks

In [0]:
%sql
-- Check Tableau data file
SELECT * FROM teams.cashalo.brett_collections_funnel_weekly_test

week_num,min_date,max_date,week_title,DPD_type,Unique_Accounts,HC_Connect,IVR_Connect,VB_Connect,Total_Connect,HC_Connect_AgentConnect,HC_Connect_DropOff,HC_Connect_Voicemail,HC_Connect_Other,AC_PTP,AC_NO_PTP,AC_Paid_Loan,AC_DNC,AC_Call_Back,AC_Hang_Up,AC_PTP_FFUP,Processed_account,HC_Account,VB_Account,HC_Spin,VB_Spin,HC_Spin_Connect,IVR_Spin_Connect,VB_Spin_Connect,HC_Spin_Connect_DropOff,HC_Spin_Connect_Voicemail,HC_Spin_Connect_Other,HC_Spin_Connect_AgentConnect,AC_Spin_PTP,AC_Spin_NO_PTP,AC_Spin_Paid_Loan,AC_Spin_DNC,AC_Spin_Call_Back,AC_Spin_Hang_Up,AC_Spin_PTP_FFUP,VB_Only_Account
48,2025-11-24,2025-11-30,Week 48: Nov-24 - Nov-30,M1,28119,1695,399,581,2675,314,849,463,69,167,15,44,11,5,39,33,237,28119,27924,1215085,1013283,3619,1193,1778,1253,1901,144,321,167,15,44,12,5,41,37,0
47,2025-11-17,2025-11-23,Week 47: Nov-17 - Nov-23,M1,29271,2268,484,732,3484,550,1101,531,86,308,20,77,20,7,31,87,425,29271,28862,1366620,966765,5047,1632,2226,1826,2445,205,571,310,20,78,20,7,32,104,0
50,2025-12-08,2025-12-14,Week 50: Dec-08 - Dec-14,Pre-Due,45314,9529,1952,1365,12846,6475,2170,715,169,5195,47,807,12,18,159,237,6061,45314,41003,1212761,687797,15244,6527,5076,4744,3489,331,6680,5200,48,858,13,25,207,329,0
48,2025-11-24,2025-11-30,Week 48: Nov-24 - Nov-30,Pre-Due,49614,8257,2211,1998,12466,5451,1906,677,223,4335,61,707,18,27,135,168,5124,49614,46416,1087448,824240,11674,5775,6119,3301,2419,393,5561,4340,62,735,20,33,153,218,0
46,2025-11-12,2025-11-16,Week 46: Nov-12 - Nov-16,M1,31880,891,519,685,2095,178,329,342,42,84,7,40,7,4,3,33,138,31880,31574,703087,727891,2493,1447,1734,728,1345,103,317,164,10,56,17,8,18,44,0
46,2025-11-12,2025-11-16,Week 46: Nov-12 - Nov-16,Pre-Due,32918,2839,1003,1519,5361,1852,576,305,106,1144,20,453,18,5,36,176,1636,32918,31575,573442,565548,7180,3731,4230,1863,1584,221,3512,2414,33,671,25,17,94,258,0
50,2025-12-08,2025-12-14,Week 50: Dec-08 - Dec-14,M1,27773,1968,362,689,3019,450,979,471,68,219,23,73,7,9,37,82,322,27773,27433,1400127,1036774,4171,1036,1821,1461,2106,151,453,219,23,73,8,9,38,83,0
49,2025-12-01,2025-12-07,Week 49: Dec-01 - Dec-07,M1,33621,2241,440,780,3461,551,1023,577,90,317,20,93,11,7,49,54,450,33621,33207,1670978,1003199,4802,1373,2065,1566,2479,199,558,317,20,93,12,8,51,57,0
47,2025-11-17,2025-11-23,Week 47: Nov-17 - Nov-23,Pre-Due,21583,4754,456,703,5913,3137,1120,385,112,2453,37,398,17,5,78,149,2905,21583,19689,490161,260039,8239,1812,2468,2445,2129,268,3397,2459,39,455,18,14,105,307,0
49,2025-12-01,2025-12-07,Week 49: Dec-01 - Dec-07,Pre-Due,37337,6380,1123,984,8487,4288,1363,539,190,3029,79,733,13,19,93,322,3944,37337,34107,828031,242224,9395,3399,2208,2650,1876,322,4547,3040,83,815,16,24,116,453,0


In [0]:
# Check buildup file
if os.path.isfile(buildup_file_name):
    tmp = pl.read_parquet("./Output/brett_collections_funnel_buildup_weekly.parquet")
    print(tmp.group_by('call_date').count().sort('call_date'))
else:
    print("No build up file")

No build up file
